In [4]:
import subprocess

# Suppress output of pip install command
subprocess.run(['pip', 'install', 'pandas', 'numpy', 'scikit-learn'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)




CompletedProcess(args=['pip', 'install', 'pandas', 'numpy', 'scikit-learn'], returncode=0)

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [6]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')


In [7]:
movies.head()

ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [8]:
movie_user_matrix = ratings.pivot_table(index='movieId', columns='userId', values='rating').fillna(0)
movie_user_matrix.head()


userId,1,2,3,4,5,6,7,8,9,10,...,659,660,661,662,663,664,665,666,667,668
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,5.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,4.0,5.0,3.0,0.0,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0
3,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity between movies
movie_similarity = cosine_similarity(movie_user_matrix)

# Check the similarity matrix shape
print(movie_similarity.shape)


(10325, 10325)


In [10]:
def recommend_movies(liked_movie_titles, top_n=5):
    liked_movie_ids = movies[movies['title'].isin(liked_movie_titles)]['movieId'].values
    liked_indices = [movie_user_matrix.index.get_loc(mid) for mid in liked_movie_ids]
    
    scores = np.zeros(movie_user_matrix.shape[0])
    
    for idx in liked_indices:
        scores += movie_similarity[idx]
    
    for idx in liked_indices:
        scores[idx] = -1  # avoid recommending already liked movies
    
    top_indices = np.argsort(scores)[::-1][:top_n]
    top_movie_ids = movie_user_matrix.index[top_indices]
    
    recommended_titles = movies[movies['movieId'].isin(top_movie_ids)]['title'].values
    return recommended_titles



In [11]:
liked_movies = ['Jumanji (1995)']

recommendations = recommend_movies(liked_movies)

print("Recommendations for you:")
for movie in recommendations:
    print("👉", movie)


Recommendations for you:
👉 Lion King, The (1994)
👉 Mask, The (1994)
👉 Speed (1994)
👉 Mrs. Doubtfire (1993)
👉 Home Alone (1990)
